In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')
DRIVE_DIR = "drive/My Drive/Springboard/capstone"

# add this to sys patch so we can import utility functions
sys.path.append(DRIVE_DIR)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, GRU, SpatialDropout1D, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
import pandas as pd
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pickle
from datetime import datetime
from sklearn.metrics import confusion_matrix, classification_report
import os
import seaborn as sns
import matplotlib.pyplot as plt
import dill


# custom utility functions
import util.dict_util as du
import util.plot_util as pu
import util.file_util as fu
import util.keras_util as ku


sns.set()

import logging
logging.basicConfig(level=logging.INFO)

In [0]:
MAX_FEATURES = 100000

In [13]:
GOOGLE_WORD2VEC_FILE = f"{DRIVE_DIR}/data/GoogleNews-vectors-negative300.bin"
tokenizer = Tokenizer(oov_token='<UNK>', num_words=MAX_FEATURES+1)
with open(GOOGLE_WORD2VEC_FILE, 'rb') as f:
    word2idx = dill.load(f)
tokenizer.word_index = word2idx

UnpicklingError: ignored

In [0]:
def load_pretrained_embeddings(word_to_index, max_features, embedding_size, embedding_file_path):    
    
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')

    # turns the embedding file into a dictionary key = word, value = word vector
    embeddings_index = dict(get_coefs(*row.split(" ")) 
                                for row in open(embedding_file_path, encoding="utf8", errors='ignore') 
                                    if len(row)>100)

    # convert the values into a array of word arrays
    all_embs = np.stack(embeddings_index.values())
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_to_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_size))
    
    for word, idx in word_to_index.items():
        if idx >= max_features: 
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[idx] = embedding_vector

    return embedding_matrix

In [0]:
EMBED_SIZE = 300 # how big is each word vector
MAX_FEATURES = 800000 # how many unique words to use (i.e num rows in embedding vector)
MAX_LEN = 200 # max number of words in a doc to use

PARAGRAM_INIT_EMBEDDINGS_FILE = f'{DRIVE_DIR}/data/cve_model_paragram_init_embeddings.pkl'

if not os.path.isfile(PARAGRAM_INIT_EMBEDDINGS_FILE):
    PARAGRAM_EMBEDDINGS_PATH = f'{DRIVE_DIR}/data/paragram_300_sl999/paragram_300_sl999.txt'
    pg_embeddings = load_pretrained_embeddings(word_to_index=word2idx, max_features=MAX_FEATURES, 
                                               embedding_size=EMBED_SIZE, 
                                               embedding_file_path=PARAGRAM_EMBEDDINGS_PATH)
    with open(PARAGRAM_INIT_EMBEDDINGS_FILE, 'wb') as f:
        dill.dump(pg_embeddings, f)
else:
    with open(PARAGRAM_INIT_EMBEDDINGS_FILE, 'rb') as f:
        pg_embeddings = dill.load(f)
        
pg_embeddings.shape 